# Set Up NVFLARE in POC Mode

[POC mode](https://nvflare.readthedocs.io/en/main/user_guide/poc_command.html) allows users to test the features of a full FLARE deployment on a single machine, without the overhead of a true distributed deployment.

Compared to the FL Simulator, where the job run is automated on a single system, POC mode allows you to establish and connect distinct server and client "systems" which can then be orchestrated using the FLARE Console.  This can be useful in preparation for a distributed deployment.

With 2.4.0 upgrade, you can also experiment your deployment options (project.yml) using POC mode. 

>It is ideal to start your NVFLARE system in POC mode from a **terminal**, not from a notebook. The terminal's virual env. must match the kernel's virtual env. In our case, we are using 'nvflare_example'.

To get started, let's look at the NVFlare CLI usage for the ``poc`` subcommand:

In [1]:

! nvflare poc -h


usage: nvflare poc [-h] [--prepare] [--start] [--stop] [--clean]
                   {prepare,prepare-examples,start,stop,clean} ...

optional arguments:
  -h, --help            show this help message and exit
  --prepare             deprecated, suggest use 'nvflare poc prepare'
  --start               deprecated, suggest use 'nvflare poc start'
  --stop                deprecated, suggest use 'nvflare poc stop'
  --clean               deprecated, suggest use 'nvflare poc clean'

poc:
  {prepare,prepare-examples,start,stop,clean}
                        poc subcommand
    prepare             prepare poc environment by provisioning local project
    prepare-examples    prepare examples
    start               start services in poc mode
    stop                stop services in poc mode
    clean               clean up poc workspace


> If you are the NVFLARE user prior to 2.4.0, you might noticed two things
> * poc command is now separate the sub-commands
> * the sub-commands such as "prepare", "start/stop" etc. are different from previous version where "--" is removed for command, such as 
>   "--prepare" becomes "prepare" and "--start" becomes "start" etc. 

We can further look at the help description for each command

> We will need ```tree``` command, so we will install ```tree`` in linux, if tree is not avaialble to you, you can replace it with 'ls -al`

In [42]:
! python -m pip install tree


[notice] A new release of pip is available: 23.0.1 -> 23.2.1
[notice] To update, run: pip install --upgrade pip


### Preparing the POC Configuration
Before running POC mode, there are a few environment variables you might consider to set. These env. variables are consider nice to have but not required since 2.4.0. We have provided other options if you preference is not use env. variables. 

##### **NVFLARE_HOME**

> Note: Set NVFLARE_HOME env. variables if only you want to use examples as jobs in POC mode.

Prior 2.4.0 release, the NVFLARE_HOME env. variable is used to find the NVFlare/exampels location. In 2.4.0, we have added 

```
nvflare poc prepare-examples <examples_location>

```
command to do so. If you decide to set it, you can set in the followings steps. set `NVFLARE_HOME` to the root of the GitHub clone.  


In [7]:
import os
NVFLARE_HOME=os.path.abspath(os.path.join(os.getcwd(), "../.."))
%env NVFLARE_HOME={NVFLARE_HOME}
! echo $NVFLARE_HOME

env: NVFLARE_HOME=/home/chester/projects/NVFlare
/home/chester/projects/NVFlare


##### **POC Workspace**

By default, POC mode uses a temporary workspace in /tmp/nvflare/poc. If you are going to use default workspace, you don't need to anything. 


If you prefer to set a workspace in different location than the default, you can change the default location with different ways.  

We have provided few ways to change the location. Prior to 2.4.0 release, you have to use environment variable NVFLARE_POC_WORKSPACE. 

* NVFLARE_POC_WORKSPACE

```
%env NVFLARE_POC_WORKSPACE=/your/new/location/path

```

Starting in 2.4.0 release, we provided another option (detailed later). But NVFLARE_POC_WORKSPACE will take precedence if both approachs are used. To set the NVFLARE_POC_WORKSPACE: 

In [14]:
%env  NVFLARE_POC_WORKSPACE=/tmp/nvflare/poc
!echo $NVFLARE_POC_WORKSPACE 

env: NVFLARE_POC_WORKSPACE=/tmp/nvflare/poc
/tmp/nvflare/poc


If you want to use the default poc workspace, ``/tmp/nvflare/poc``, you don't need set ``NVFLARE_POC_WORKSPACE``. 

* Config command

If you prefer not to use environment variable, you can do the followings: 

```
! nvflare config -pw /tmp/nvflare/poc

```
or 
```
! nvflare config -poc_workspace_dir /tmp/nvflare/poc
```

In [ ]:
! nvflare config -pw /tmp/nvflare/poc

Look at the nvflare config command

In [16]:
! nvflare config -h

usage: nvflare config [-h] [-d [STARTUP_KIT_DIR]] [-pw [POC_WORKSPACE_DIR]]
                      [-debug]

optional arguments:
  -h, --help            show this help message and exit
  -d [STARTUP_KIT_DIR], --startup_kit_dir [STARTUP_KIT_DIR]
                        startup kit location
  -pw [POC_WORKSPACE_DIR], --poc_workspace_dir [POC_WORKSPACE_DIR]
                        POC Workspace location
  -debug, --debug       debug is on


This should set the default variable to the following location: 


Once can set the POC_WORKSPACE_DIR as well as other location as well. 

* Hidden Config

What config command does is to write the POC workspace directory to a hidden configuration file. 

```
~/.nvflare/config.conf
```

You can also edit directly the config file without using the config command. 

In [13]:
!cat ~/.nvflare/config.conf

startup_kit {
  path = "/tmp/nvflare/poc1/example_project/prod_00"
}
poc_workspace {
  path = "/tmp/nvflare/poc1"
}
job_template {
  path = "/home/chester/projects/NVFlare/job_templates"
}


### Preparing the POC workspace

Now that we've configured out POC configuration, we can prepare the POC workspace.  By default, the following command will generate POC packages for a server and two clients.
nvflare poc prepare
```
    nvflare poc prepare 
```

You can specify a different number of clients with the -n option:

```
   nvflare poc prepare -n N
```

If you running the POC prepare command, there is a prompt for user to answer.

#### Various Prepare Options

In [5]:
!  nvflare poc prepare -h

usage: nvflare poc prepare [-h] [-n [NUMBER_OF_CLIENTS]]
                           [-c [CLIENTS [CLIENTS ...]]] [-e [EXAMPLES]] [-he]
                           [-i [PROJECT_INPUT]] [-d [DOCKER_IMAGE]] [-debug]

optional arguments:
  -h, --help            show this help message and exit
  -n [NUMBER_OF_CLIENTS], --number_of_clients [NUMBER_OF_CLIENTS]
                        number of sites or clients, default to 2
  -c [CLIENTS [CLIENTS ...]], --clients [CLIENTS [CLIENTS ...]]
                        Space separated client names. If specified,
                        number_of_clients argument will be ignored.
  -e [EXAMPLES], --examples [EXAMPLES]
                        examples directory
  -he, --he             enable homomorphic encryption.
  -i [PROJECT_INPUT], --project_input [PROJECT_INPUT]
                        project.yaml file path, If specified,
                        'number_of_clients','clients' and 'docker' specific
                        options will be ignored.
  

We have many options to prepare the POC workspace. let's look at some of them

* use client's site name (instead of use default site-1,2 etc.)

```
! nvflare poc prepare -c hospital-1 hospital-2
```
This will create two clients named **hospital-1** and **hospital-2**

* -d docker_image

This instruct the provision tool to create a docker.sh use the specified docker_image. The docker.sh will start in detached mode. 

* -i project.yml 

This optional allows you experiment the your project.yml locally, try local before used in production. 

* All specified options will resulting an project.yml file located at 

<poc_workspace>/project.yml 

for example, ```/tmp/nvflare/poc/project.yml```


Let's try out some of options

* **Named Clients**

instead of using default  site-1, site-2 etc. user can specified the client names

In [18]:
! nvflare poc prepare -c hospital-1 hospital-2


prepare poc at /tmp/nvflare/poc for 2 clients
provision at /tmp/nvflare/poc for 2 clients with /tmp/nvflare/poc/project.yml
Generated results can be found under /tmp/nvflare/poc/example_project/prod_00. 
link examples from /home/chester/projects/NVFlare/examples to /tmp/nvflare/poc/example_project/prod_00/admin@nvidia.com/transfer


Now, let's look at the result, if you have ```tree`` command installed in linux (``` pip install tree```), you can use it. otherwise, use ```ls -al```. 

In [19]:
! tree /tmp/nvflare/poc/example_project/prod_00

/tmp/nvflare/poc/example_project/prod_00
├── admin@nvidia.com
│   ├── local
│   ├── startup
│   │   ├── client.crt
│   │   ├── client.key
│   │   ├── client.pfx
│   │   ├── fed_admin.json
│   │   ├── fl_admin.sh
│   │   ├── readme.txt
│   │   └── rootCA.pem
│   └── transfer -> /home/chester/projects/NVFlare/examples
├── hospital-1
│   ├── local
│   │   ├── authorization.json.default
│   │   ├── log.config.default
│   │   ├── privacy.json.sample
│   │   └── resources.json.default
│   ├── readme.txt
│   ├── startup
│   │   ├── client.crt
│   │   ├── client.key
│   │   ├── client.pfx
│   │   ├── fed_client.json
│   │   ├── rootCA.pem
│   │   ├── signature.json
│   │   ├── start.sh
│   │   ├── stop_fl.sh
│   │   └── sub_start.sh
│   └── transfer
├── hospital-2
│   ├── local
│   │   ├── authorization.json.default
│   │   ├── log.config.default
│   │   ├── privacy.json.sample
│   │   └── resources.json.default
│   ├── readme.txt
│   ├── startup
│   │   ├── client.crt
│   │   ├── client.key
│

Now two clients are named "hospital-1" and "hospital-2". 

* **Setup NVFLARE in docker mode**

Normally, when we using ```nvflare poc start```, the clients and server are started as process in the local host. 
What if you would like to experiment runing client and server in dockers ? You can do it now, 

```
nvflare poc prepare -n 2 -d  <docker_image>

```

This will create docker.sh for each client and server which will run docker pull docker_image from docker hub, then running the docker in detached mode. The docker_image must use nvflare docker must have flare pre-installed. For example, we use the following docker image "nvflare/nvflare"
> Note: you should build your own image if you want to include your dependencies such as pytorch etc.


In [28]:
! echo 'y'| nvflare poc prepare -d 'nvflare/nvflare'

prepare poc at /tmp/nvflare/poc for 2 clients
This will delete poc folder in /tmp/nvflare/poc directory and create a new one. Is it OK to proceed? (y/N) provision at /tmp/nvflare/poc for 2 clients with /tmp/nvflare/poc/project.yml
Generated results can be found under /tmp/nvflare/poc/example_project/prod_00. 
link examples from /home/chester/projects/NVFlare/examples to /tmp/nvflare/poc/example_project/prod_00/admin@nvidia.com/transfer


In [29]:
! tree /tmp/nvflare/poc/example_project/prod_00

/tmp/nvflare/poc/example_project/prod_00
├── admin@nvidia.com
│   ├── local
│   ├── startup
│   │   ├── client.crt
│   │   ├── client.key
│   │   ├── client.pfx
│   │   ├── docker.sh
│   │   ├── fed_admin.json
│   │   ├── fl_admin.sh
│   │   ├── readme.txt
│   │   └── rootCA.pem
│   └── transfer -> /home/chester/projects/NVFlare/examples
├── server
│   ├── local
│   │   ├── authorization.json.default
│   │   ├── log.config.default
│   │   ├── privacy.json.sample
│   │   └── resources.json.default
│   ├── readme.txt
│   ├── startup
│   │   ├── docker.sh
│   │   ├── fed_server.json
│   │   ├── rootCA.pem
│   │   ├── server.crt
│   │   ├── server.key
│   │   ├── server.pfx
│   │   ├── signature.json
│   │   ├── start.sh
│   │   ├── stop_fl.sh
│   │   └── sub_start.sh
│   └── transfer
├── site-1
│   ├── local
│   │   ├── authorization.json.default
│   │   ├── log.config.default
│   │   ├── privacy.json.sample
│   │   └── resources.json.default
│   ├── readme.txt
│   ├── startup
│   │   ├──

Notice the new "docker.sh", now each client will using docker run with we specified "nvflare/nvflare" docker image

In [ ]:
* **simulate production deployment setup locally**

Assuming we have a new custom project.yml, we like to test out the project.yml locally before we do the real provision. 

We can actually do it with POC !


```
nvflare poc prepare -i <your project.yml>
```

Let's try this out.  we have prepared an custom_project.yml file: 


In [37]:
!cat ./custom_project.yml

api_version: 3
builders:
- args:
    template_file: master_template.yml
  path: nvflare.lighter.impl.workspace.WorkspaceBuilder
- path: nvflare.lighter.impl.template.TemplateBuilder
- args:
    config_folder: config
    overseer_agent:
      args:
        sp_end_point: server:8002:8003
      overseer_exists: false
      path: nvflare.ha.dummy_overseer_agent.DummyOverseerAgent
  path: nvflare.lighter.impl.static_file.StaticFileBuilder
- path: nvflare.lighter.impl.cert.CertBuilder
- path: nvflare.lighter.impl.signature.SignatureBuilder
description: health_project project yaml file
name: health_project
participants:
- admin_port: 8003
  fed_learn_port: 8002
  name: general-hospital-server
  org: nonprofit_health
  type: server
- name: admin@nvidia.com
  org: nonprofit_health
  role: project_admin
  type: admin
- name: us-hospital
  org: nonprofit_health
  type: client
- name: europe-hospital
  org: nonprofit_health
  type: client


In [41]:
! echo 'y' | nvflare poc prepare -i ./custom_project.yml

prepare poc at /tmp/nvflare/poc with ./custom_project.yml
This will delete poc folder in /tmp/nvflare/poc directory and create a new one. Is it OK to proceed? (y/N) provision at /tmp/nvflare/poc for 2 clients with ./custom_project.yml
Generated results can be found under /tmp/nvflare/poc/health_project/prod_00. 
link examples from /home/chester/projects/NVFlare/examples to /tmp/nvflare/poc/health_project/prod_00/admin@nvidia.com/transfer


In [43]:
! tree /tmp/nvflare/poc/health_project/prod_00

/tmp/nvflare/poc/health_project/prod_00
├── admin@nvidia.com
│   ├── local
│   ├── startup
│   │   ├── client.crt
│   │   ├── client.key
│   │   ├── client.pfx
│   │   ├── fed_admin.json
│   │   ├── fl_admin.sh
│   │   ├── readme.txt
│   │   └── rootCA.pem
│   └── transfer -> /home/chester/projects/NVFlare/examples
├── europe-hospital
│   ├── local
│   │   ├── authorization.json.default
│   │   ├── log.config.default
│   │   ├── privacy.json.sample
│   │   └── resources.json.default
│   ├── readme.txt
│   ├── startup
│   │   ├── client.crt
│   │   ├── client.key
│   │   ├── client.pfx
│   │   ├── fed_client.json
│   │   ├── rootCA.pem
│   │   ├── signature.json
│   │   ├── start.sh
│   │   ├── stop_fl.sh
│   │   └── sub_start.sh
│   └── transfer
├── general-hospital-server
│   ├── local
│   │   ├── authorization.json.default
│   │   ├── log.config.default
│   │   ├── privacy.json.sample
│   │   └── resources.json.default
│   ├── readme.txt
│   ├── startup
│   │   ├── fed_server.json
│ 

### Prepare examples

If you did not set the NVFLARE_HOME env. variable, or did not fork the NVFlare Github Repo, or you want to change the examples directory 

you can use the following command to setup examples

```
! nvflare poc prepare-examples <examples directory>
```

### Start NVFlare system in POC mode

When starting the POC deployment, it's necessary to use a separate terminal since the `nvflare poc start` command will run  in the foreground emitting output from the server and any connected clients.

Also note that `nvflare poc start` starts all participants, including the admin console. We want to start the 2 clients and 1 server, but without the FLARE console (aka Admin Console), as we are going to interact with the system from a notebook.

Create a terminal to execute the following command (in JupyterLab or host terminal), then

```
    nvflare poc start -ex admin@nvidia.com
```
**Note:**
    Using ```%%bash -bg``` to run the above command in a code cell may not always work

### Check system information

In [ ]:

import os
from nvflare.fuel.flare_api.flare_api import new_secure_session

poc_workspace = os.getenv('NVFLARE_POC_WORKSPACE')

workspace = "/tmp/nvflare/poc" if  poc_workspace is None else poc_workspace

default_poc_prepared_dir = os.path.join(workspace, "example_project/prod_00")
admin_dir = os.path.join(default_poc_prepared_dir, "admin@nvidia.com")
sess = new_secure_session("admin@nvidia.com", admin_dir)
sys_info = sess.get_system_info()

print(f"Server info:\n{sys_info.server_info}")
print("\nClient info")
for client in sys_info.client_info:
    print(client)

Once the system is up, we are now ready to go back to the notebook to work on your project.

### Stop FLARE system in POC

Once you are done with the FLARE system, you can shut down the poc system.
From terminal, issue the following command stop poc in the terminal
```
   nvflare poc stop
```


### Clean up POC workspace

If you want to clean up the POC workspace and delete the poc workspace directory containing all packages you may have prepared, you can use the following command:

In [ ]:
! nvflare poc clean

You can check if the nvflare system is shutdown cleanly with:

```
ps -eaf | grep nvflare
```

If you see output like the following, then nvflare systems are still running:

```
510535    1932  1 18:54 pts/1    00:00:03 python3 -u -m nvflare.private.fed.app.client.client_train -m /tmp/workspace/example_project/prod_00/site-1/startup/.. -s fed_client.json --set secure_train=true uid=site-1 org=nvidia config_folder=config
510539    1932  1 18:54 pts/1    00:00:03 python3 -u -m nvflare.private.fed.app.client.client_train -m /tmp/workspace/example_project/prod_00/site-2/startup/.. -s fed_client.json --set secure_train=true uid=site-2 org=nvidia config_folder=config
510543    1932  1 18:54 pts/1    00:00:04 python3 -u -m nvflare.private.fed.app.server.server_train -m /tmp/workspace/example_project/prod_00/localhost/startup/.. -s fed_server.json --set secure_train=true org=nvidia config_folder=config
```

If you have already used ``nvflare poc clean`` to delete the workspace, you may need to kill the processes manually.


### After POC

Once we experimented with different options, we are ready to move on to the production beyond local host. 
You don't need to manual edit the project.yml to do so. The resulting project.yml is auto-generated for you and located at 

```
  <POC_Workspace>/project.yml
```

For example,


In [48]:
! echo 'y' | nvflare poc prepare -c hospital_1 hospital_2 -d 'nvflare/nvflare' -he  

prepare poc at /tmp/nvflare/poc for 2 clients
This will delete poc folder in /tmp/nvflare/poc directory and create a new one. Is it OK to proceed? (y/N) provision at /tmp/nvflare/poc for 2 clients with /tmp/nvflare/poc/project.yml
Generated results can be found under /tmp/nvflare/poc/example_project/prod_00. 
link examples from /home/chester/projects/NVFlare/examples to /tmp/nvflare/poc/example_project/prod_00/admin@nvidia.com/transfer


In [49]:
! cat  /tmp/nvflare/poc/project.yml

api_version: 3
builders:
- args:
    template_file: master_template.yml
  path: nvflare.lighter.impl.workspace.WorkspaceBuilder
- path: nvflare.lighter.impl.template.TemplateBuilder
- args:
    config_folder: config
    docker_image: nvflare/nvflare
    overseer_agent:
      args:
        sp_end_point: server:8002:8003
      overseer_exists: false
      path: nvflare.ha.dummy_overseer_agent.DummyOverseerAgent
  path: nvflare.lighter.impl.static_file.StaticFileBuilder
- path: nvflare.lighter.impl.cert.CertBuilder
- path: nvflare.lighter.impl.signature.SignatureBuilder
- args: {}
  path: nvflare.lighter.impl.he.HEBuilder
description: NVIDIA FLARE sample project yaml file
name: example_project
participants:
- admin_port: 8003
  fed_learn_port: 8002
  name: server
  org: nvidia
  type: server
- name: admin@nvidia.com
  org: nvidia
  role: project_admin
  type: admin
- name: hospital_1
  org: nvidia
  type: client
- name: hospital_2
  org: nvidia
  type: client
